# Classification d'images

## Importer TensorFlow et d'autres bibliothèques

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib


## Chargement des données

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

folderName = 'datasets/garbage_ds'

# Données d'entrainement
train_ds = tf.keras.utils.image_dataset_from_directory(
  folderName,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Données de validation
val_ds = tf.keras.utils.image_dataset_from_directory(
  folderName,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


class_names = train_ds.class_names
print(class_names)

## Visualisation des données

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

## Standardiser les données

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

## Configurer l'ensemble de données pour les performances

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Création du modèle

In [ ]:
# Augmentation virtuelle des données d'entrainement
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
# Visualisation des données "augmentées"
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[10].numpy().astype("uint8"))
    plt.axis("off")

In [136]:
# Nombre de classes d'éléments à détecter
num_classes = len(class_names)

model = tf.keras.Sequential(name="ALYRA-IA-DEV-PROJET-E1-EDOUNZE-CHARLES")

# Cette première couche applique des transformations aléatoires aux images d'entrée (comme la rotation, le zoom, le décalage) 
# pour augmenter la diversité du jeu de données d'entraînement sans avoir besoin de plus d'images. Cela aide le modèle à généraliser mieux à partir de données limitées.
model.add(data_augmentation)

# Cette couche normalise les images d'entrée en divisant les valeurs des pixels par 255. 
# Cela transforme les valeurs de pixels de la plage [0, 255] à [0, 1], ce qui est une pratique courante pour faciliter l'apprentissage du modèle.
model.add(tf.keras.layers.Rescaling(1./255))

# Première couche de convolution qui applique 32 filtres de 3x3 sur l'image d'entrée pour extraire des caractéristiques de bas niveau comme les bords. 
# La fonction d'activation 'relu' ajoute de la non-linéarité, permettant à la couche d'apprendre des motifs complexes.
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))

# Ces couches suivent généralement les couches Conv2D et servent à réduire la dimensionnalité spatiale de la couche Conv2D précédente, en conservant les caractéristiques les plus importantes. 
# Elles aident à rendre le modèle plus efficace et à diminuer le risque de surajustement (overfitting) en extrayant les caractéristiques les plus saillantes.
model.add(tf.keras.layers.MaxPooling2D())

# Deuxième couche de convolution qui applique à nouveau 32 filtres de 3x3. 
# À ce stade, le modèle cherche à extraire des caractéristiques plus complexes à partir des informations simplifiées par la première couche de MaxPooling.
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))

# Agit sur la sortie de la deuxième couche Conv2D pour réduire encore la dimensionnalité. 
# Cette étape continue de condenser l'information, permettant au modèle de devenir progressivement plus abstrait et concentré sur les caractéristiques saillantes (remarquables).
model.add(tf.keras.layers.MaxPooling2D())

# Troisième couche de convolution identique aux précédentes en termes de nombre de filtres et de taille, 
# poussant le modèle à extraire et à apprendre des caractéristiques encore plus abstraites des données
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))

# Appliquée après la troisième couche Conv2D, cette couche de pooling continue de réduire la dimensionnalité de la représentation de l'image, 
# préparant les données pour une analyse de haut niveau dans les couches suivantes.
tf.keras.layers.MaxPooling2D(),

# Cette couche ignore aléatoirement 20% des neurones durant l'entraînement, réduisant ainsi le surajustement en forçant 
# le modèle à apprendre des caractéristiques plus robustes qui ne dépendent pas d'un petit nombre de neurones.
model.add(layers.Dropout(0.2))

# Cette couche aplatie les matrices multidimensionnelles en vecteurs unidimensionnels, 
# permettant de passer de représentations spatiales à un format compatible avec les couches denses.
model.add(tf.keras.layers.Flatten())

# Une couche dense (ou entièrement connectée) qui a 128 neurones et utilise ReLU comme fonction d'activation. 
# Cette couche permet au modèle de combiner les caractéristiques apprises par les couches précédentes pour former des abstractions de plus haut niveau.
model.add(tf.keras.layers.Dense(128, activation='relu'))

# La dernière couche dense avec un nombre de neurones égal au nombre de classes dans le jeu de données. 
# Elle génère la sortie du modèle, où chaque neurone représente la probabilité que l'image d'entrée appartienne à une classe spécifique.
model.add(tf.keras.layers.Dense(num_classes))

model = tf.keras.Sequential([
  # Cette première couche applique des transformations aléatoires aux images d'entrée (comme la rotation, le zoom, le décalage) 
  # pour augmenter la diversité du jeu de données d'entraînement sans avoir besoin de plus d'images. Cela aide le modèle à généraliser mieux à partir de données limitées.
  data_augmentation,
  
  # Cette couche normalise les images d'entrée en divisant les valeurs des pixels par 255. 
  # Cela transforme les valeurs de pixels de la plage [0, 255] à [0, 1], ce qui est une pratique courante pour faciliter l'apprentissage du modèle.
  tf.keras.layers.Rescaling(1./255),

  # Plusieurs de ces couches appliquent des filtres (ou noyaux) pour capturer des caractéristiques spatiales dans les images. 
  # Chaque couche Conv2D ici utilise 32 filtres de taille 3x3 et une fonction d'activation ReLU pour introduire des non-linéarités. 
  # Ces couches aident le modèle à apprendre différents aspects visuels des images, comme les bords, les textures, ou d'autres motifs.
  tf.keras.layers.Conv2D(32, 3, activation='relu'),

  # Ces couches suivent généralement les couches Conv2D et servent à réduire la dimensionnalité spatiale des représentations d'entrée. 
  # Elles aident à rendre le modèle plus efficace et à diminuer le risque de surajustement en extrayant les caractéristiques les plus saillantes.
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Conv2D(32, 3, activation='relu'),

  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Conv2D(32, 3, activation='relu'),

  tf.keras.layers.MaxPooling2D(),

  # Cette couche ignore aléatoirement 20% des neurones durant l'entraînement, réduisant ainsi le surajustement en forçant 
  # le modèle à apprendre des caractéristiques plus robustes qui ne dépendent pas d'un petit nombre de neurones.
  layers.Dropout(0.2),

  # Cette couche aplatie les matrices multidimensionnelles en vecteurs unidimensionnels, 
  # permettant de passer de représentations spatiales à un format compatible avec les couches denses.
  tf.keras.layers.Flatten(),

  # Une couche dense (ou entièrement connectée) qui a 128 neurones et utilise ReLU comme fonction d'activation. 
  # Cette couche permet au modèle de combiner les caractéristiques apprises par les couches précédentes pour former des abstractions de plus haut niveau.
  tf.keras.layers.Dense(128, activation='relu'),

  #  La dernière couche dense avec un nombre de neurones égal au nombre de classes dans le jeu de données. 
  # Elle génère la sortie du modèle, où chaque neurone représente la probabilité que l'image d'entrée appartienne à une classe spécifique.
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [ ]:
model.summary()

## Entrainement du modèle

In [ ]:
epochs=15

history  = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Résultats de l'entrainement

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Prédiction de nouvelles données

In [ ]:
object_url = "https://www.francetvinfo.fr/pictures/WCd2DGgauBIkM2KDQcLfNrjIois/1500x843/2023/01/30/63d7748135eff_maxnewsfrfour319227.jpg"
object_path = tf.keras.utils.get_file('Object_image', origin=object_url)

img = tf.keras.utils.load_img(
    object_path, target_size=(img_height, img_width)
)

print(type(img))

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)